### Chapter 15: Advanced RAG 구축 -1 multiquery + unique-union

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='\n\n"소득·집값도 안 본다"…초유의 주담대 절대한도, 왜 6억원일까\n\n금융당국이 소득 상환 능력, 주택담보대출비율(LTV) 규제를 넘어선 \'주담대 최대 6억 원 제한\'이란 초강력 규제를 들고 나왔다. 집값이 급등하던 2019년 15억 원 이상 주택에 대해 대출을\n\n\n뉴스1\n\n\n\n\n70\n개의 관련뉴스 더보기\n\n\n\n\n\n中企 7월 경기전망지수 76.6…전월比 1.6p 상승\n\n국내 중소기업들의 7월 경기전망이 6월보다 개선된 것으로 나타났다. 제조업의 경기전망지수는 지난달보다 하락했으나, 비제조업 경기전망지수가 개선되면서 이를 상쇄시켰다. 업황전망 SBHI(이미지=중기중앙회) 29일 중소\n\n\n이데일리\n\n\n\n\n21\n개의 관련뉴스 더보기\n\n\n\n\n\n6월 가계대출 증가액 이미 5조8천억…비대면 대출 중단\n\n6월 가계대출 증가액이 5개월 연속 확대돼 7조원대에 이를지 주목된다. 은행들은 지난 27일 정부가 발표한 ‘가계부채 관리 강화 방안’을 전산에 반영하기 위해 비대면 대출을 일제히 중단한 상태다. 다만 은행들은 발표\n\n\n한겨레\n\n\n\n\n33\n개의 관련뉴스 더보기\n\n\n\n\n\n"제네시스 청주, 단순 전시장 넘어 고객과 교감하는 플랫폼으로"\n\n제네시스 최대 규모…디자인 추천·공예 전시·시승 프로그램 눈길 "제네시스 청주는 단순한 자동차 전시장을 넘어 고객과 깊이 교감하는 커뮤니티 플랫폼으로 나아가려 합니다." 문정균 제네시스 공간경험실장은 지난 25일 충\n\n\n연합뉴스\n\n\n\n\n16\n개의 관련뉴스 더보기\n\n\n\n\n\n신한은행 배달플랫폼 \'땡겨요\'에 스타벅스 입점\n\n신한은행은 신세계그룹과 서울시 중구 소재 본점에서 신사업 지원 및 동반성장 생태계 구축을 위한 업무협약을 체결했다고 29일 밝혔다. 두 회사는 이번

In [5]:
# Split
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
# splits = text_splitter.split_documents(docs)

#이점: from_tiktoken_encoder 메서드는 텍스트를 분할할 때 tiktoken 인코더를 사용하여 텍스트를 토큰으로 변환한 다음 분할합니다. 이 방법은 특히 OpenAI 모델과 같은 토큰 기반 언어 모델에서 텍스트를 처리할 때 유용합니다. 
# 토큰 단위로 정확한 분할을 가능하게 하므로, 텍스트가 모델의 입력 토큰 제한에 맞도록 더 정밀하게 분할할 수 있습니다.
# 적용 상황: 이 방법은 모델이 토큰 수를 기준으로 텍스트를 처리해야 할 때(예: GPT 모델) 유리합니다. 특히 텍스트가 다양한 언어를 포함하거나 복잡한 구문을 가질 때 유용합니다.

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='"소득·집값도 안 본다"…초유의 주담대 절대한도, 왜 6억원일까\n\n금융당국이 소득 상환 능력, 주택담보대출비율(LTV) 규제를 넘어선 \'주담대 최대 6억 원 제한\'이란 초강력 규제를 들고 나왔다. 집값이 급등하던 2019년 15억 원 이상 주택에 대해 대출을\n\n\n뉴스1'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='뉴스1\n\n\n\n\n70\n개의 관련뉴스 더보기\n\n\n\n\n\n中企 7월 경기전망지수 76.6…전월比 1.6p 상승'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='국내 중소기업들의 7월 경기전망이 6월보다 개선된 것으로 나타났다. 제조업의 경기전망지수는 지난달보다 하락했으나, 비제조업 경기전망지수가 개선되면서 이를 상쇄시켰다. 업황전망 SBHI(이미지=중기중앙회) 29일 중소\n\n\n이데일리'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='이데일리\n\n\n\n\n21\n개의 관련뉴스 더보기\n\n\n\n\n\n6월 가계대출 증가액 이미 5조8천억…비대면 대출 중단'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='6월 가계대출 증가액이 5개월 연속 확대돼 7조원대에 이를지 주목된다. 은행들은 지난 27일 정부가 발표한 ‘가계부채 관리 강화 방안’을 전산에 반영하기 위해 비대면 대출을 일제히 중단한 상태다. 다만 은행들은 발표\n\n\n한겨레\n\n\n\n\n33\

In [6]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

In [7]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# 그러나 최종적으로 상위 1개 문서만 반환합니다
retriever = vectorstore.as_retriever(
    search_type="mmr", # MMR 알고리즘을 사용하여 검색
    search_kwargs={'k':1,'fetch_k':4} # 상위 1개의 문서를 반환하지만, 고려할 문서는 4개로 설정
)

In [8]:
# Prompt
prompt = hub.pull("sungwoo/ragbasic")

In [9]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("국채 관련한 정보를 알려줘")


'국채에 대한 구체적인 정보는 제공되지 않았습니다. 현재 검색 결과는 가계부채 관리 강화 방안과 관련된 내용입니다. 국채에 대한 추가 정보가 필요하다면 다른 출처를 참고하시기 바랍니다.'

In [13]:
docs =retriever.invoke("국채 관련한 정보를 알려줘")
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='전격 시행된 정부의 ‘가계부채 관리 강화 방안’에 따라 29일 서울 부동산 시장은 충격에 빠진 모습이다. 시장에선 이번 대책의 파급 효과, 후속 대책 여부 등을 지켜보기 위해 당분간 매도·매수자들이 관망하면서 매매거\n\n\n한겨레\n\n3시간전')]

In [60]:
# # 문서 검색 시 더 높은 다양성을 가진 문서를 더 많이 검색합니다.
# # 데이터셋에 유사한 문서가 많을 경우 유용합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR(Maximal Marginal Relevance) 알고리즘을 사용하여 검색
#     search_kwargs={'k': 6, 'lambda_mult': 0.25}  # 상위 6개의 문서를 검색하고 다양성을 높이기 위해 lambda 값을 0.25로 설정
# )

# # MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# # 그러나 최종적으로 상위 5개 문서만 반환합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR 알고리즘을 사용하여 검색
#     search_kwargs={'k': 5, 'fetch_k': 50}  # 상위 5개의 문서를 반환하지만, 고려할 문서는 50개로 설정
# )

# # 특정 임계값 이상의 유사도 점수를 가진 문서만 검색합니다.
# docsearch.as_retriever(
#     search_type="similarity_score_threshold",  # 유사도 점수 기반 검색
#     search_kwargs={'score_threshold': 0.8}  # 유사도 점수가 0.8 이상인 문서만 검색
# )

# # 데이터셋에서 가장 유사한 문서 하나만 검색합니다.
# docsearch.as_retriever(search_kwargs={'k': 1})

### multiquery + unique-union

# Multi Query
![Example Image](./advancedRagFlow.png)

In [14]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)


In [15]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [16]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five 
# different versions of the given user question to retrieve relevant documents from a vector 
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search. 
# Provide these alternative questions separated by newlines. Original question: {question}"""
# prompt_perspectives = ChatPromptTemplate.from_template(template)

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다. 
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다. 
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
generated_query

['집값의 미래 전망은 어떻게 될까요?  ',
 '현재 집값의 추세와 앞으로의 변화는 어떤 영향을 받을까요?  ',
 '부동산 시장에서 집값이 오를지 내릴지에 대한 예측은 무엇인가요?  ',
 '향후 몇 년간 집값의 변동성에 대해 어떤 의견이 있나요?  ',
 '집값 상승 또는 하락의 주요 요인은 무엇이라고 생각하시나요?  ']

In [17]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ 고유한 문서들의 합집합을 생성하는 함수입니다. """
    
    # 리스트의 리스트를 평탄화하고, 각 문서를 문자열로 직렬화합니다.
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    # 중복된 문서를 제거하고 고유한 문서만 남깁니다.
    unique_docs = list(set(flattened_docs))
    
    # 고유한 문서를 원래의 문서 객체로 변환하여 반환합니다.
    return [loads(doc) for doc in unique_docs]

In [18]:
# 사용자 질문 정의
question = "집값의 향방?"

In [19]:
# 문서 검색 체인을 구성합니다.
# generate_queries: 주어진 질문에 대해 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리를 바탕으로 관련 문서를 검색합니다.
# get_unique_union: 검색된 문서에서 중복을 제거하고 고유한 문서들을 반환합니다.
retrieval_chain = generate_queries | retriever.map() | get_unique_union


In [20]:
# 체인을 실행하여 질문에 대한 관련 문서를 검색하고 고유한 문서를 반환합니다.
docs = retrieval_chain.invoke({"question": question})


In [21]:
# 검색된 고유 문서들의 개수를 출력합니다.
len(docs)

# 검색된 고유 문서들을 출력합니다.
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='서울 아파트값 급등에 대응해 정부가 대대적인 대출 조이기에 나선 가운데 정책대출 한도도 크게 줄어들 전망이다. 윤석열 정부 당시 정책대출 수혜 대상을 크게 확대하면서 집값 상승의 불씨를 댕겼다는 평가를 받았는데, 다\n\n\n한겨레\n\n3시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난 28일부터 정부의 가계부채 관리방안에 따라 주택담보대출 규제가 시행되면서 최근 매매가격이 급등하던 서울 마포·성동구 등의 아파트 거래가 주춤하는 분위기다. 강력한 대출 규제로 급한 불 끄기에 나선 정부는 시장 \n\n\n경향신문\n\n4시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='전격 시행된 정부의 ‘가계부채 관리 강화 방안’에 따라 29일 서울 부동산 시장은 충격에 빠진 모습이다. 시장에선 이번 대책의 파급 효과, 후속 대책 여부 등을 지켜보기 위해 당분간 매도·매수자들이 관망하면서 매매거\n\n\n한겨레\n\n3시간전')]

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

'현재 서울 아파트값은 급등세를 보이고 있으며, 정부는 이에 대응하기 위해 대출 규제를 강화하고 있습니다. 정책대출 한도가 줄어들 것으로 예상되며, 이는 집값 상승의 원인으로 지적된 바 있습니다. 또한, 정부의 고강도 부동산 대책이 발표되면서 부산을 포함한 지방 광역시의 부동산 시장도 위축될 것으로 우려되고 있습니다. 이러한 대출 규제와 정부의 강력한 시그널은 시장에 큰 영향을 미치고 있으며, 매도·매수자들이 관망하는 상황이 이어지고 있습니다. 따라서 집값의 향방은 정부의 정책과 시장 반응에 따라 변동성이 클 것으로 보입니다.'